In [1]:
import json
import re

import numpy as np 
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset=pd.read_csv("/content/products.csv")

In [3]:
dataset

,Product Name,Price,Features,Rating,Review
0,Dell Vostro Core i5 10th Gen - (8 GB/1 TB HDD/...,"₹53,490",Intel Core i5 Processor (10th Gen)8 GB DDR4 RA...,4.2,It's been 2 days since the purchase of this pr...
1,Dell Vostro Core i3 10th Gen - (4 GB/1 TB HDD/...,"₹34,766",Intel Core i3 Processor (10th Gen)4 GB DDR4 RA...,3.7,Not good at all... Vision is also not clear an...
2,Dell Vostro Core i3 10th Gen - (4 GB/1 TB HDD/...,"₹39,998",Intel Core i3 Processor (10th Gen)4 GB DDR4 RA...,4.3,After 1 week of usage the laptop performance i...
3,Dell Vostro Core i3 10th Gen - (4 GB/256 GB SS...,"₹35,990",Intel Core i3 Processor (10th Gen)4 GB DDR4 RA...,4.2,The best config for kids. Runs most of the gam...
4,Dell Inspiron 7000 Core i5 10th Gen - (8 GB/51...,"₹91,252",Intel Core i5 Processor (10th Gen)8 GB DDR4 RA...,0.0,No reviews for this Laptop
5,Dell G3 Core i7 9th Gen - (8 GB/1 TB HDD/512 G...,"₹84,990",Intel Core i7 Processor (9th Gen)8 GB DDR4 RAM...,4.4,Display is 220nits bright( Sufficient for dail...
6,Dell Inspiron 3000 Core i7 9th Gen - (8 GB/512...,"₹93,490",Intel Core i7 Processor (9th Gen)8 GB DDR4 RAM...,2.7,No reviews for this Laptop
7,Dell Inspiron Core i3 10th Gen - (4 GB/1 TB HD...,"₹62,990",Intel Core i3 Processor (10th Gen)4 GB DDR4 RA...,4.5,Too good laptop look wise performance wise ver...
8,Dell Inspiron Core i5 10th Gen - (8 GB/512 GB ...,"₹86,990",Intel Core i5 Processor (10th Gen)8 GB DDR4 RA...,0.0,No reviews for this Laptop
9,Dell Vostro 3000 Core i5 8th Gen - (8 GB/1 TB ...,"₹51,130",Intel Core i5 Processor (8th Gen)8 GB DDR4 RAM...,4.1,this is good product. i got it in sell at quit...


In [4]:
dataset["Features"] = dataset["Features"].str.lower()


In [5]:
dataset

,Product Name,Price,Features,Rating,Review
0,Dell Vostro Core i5 10th Gen - (8 GB/1 TB HDD/...,"₹53,490",intel core i5 processor (10th gen)8 gb ddr4 ra...,4.2,It's been 2 days since the purchase of this pr...
1,Dell Vostro Core i3 10th Gen - (4 GB/1 TB HDD/...,"₹34,766",intel core i3 processor (10th gen)4 gb ddr4 ra...,3.7,Not good at all... Vision is also not clear an...
2,Dell Vostro Core i3 10th Gen - (4 GB/1 TB HDD/...,"₹39,998",intel core i3 processor (10th gen)4 gb ddr4 ra...,4.3,After 1 week of usage the laptop performance i...
3,Dell Vostro Core i3 10th Gen - (4 GB/256 GB SS...,"₹35,990",intel core i3 processor (10th gen)4 gb ddr4 ra...,4.2,The best config for kids. Runs most of the gam...
4,Dell Inspiron 7000 Core i5 10th Gen - (8 GB/51...,"₹91,252",intel core i5 processor (10th gen)8 gb ddr4 ra...,0.0,No reviews for this Laptop
5,Dell G3 Core i7 9th Gen - (8 GB/1 TB HDD/512 G...,"₹84,990",intel core i7 processor (9th gen)8 gb ddr4 ram...,4.4,Display is 220nits bright( Sufficient for dail...
6,Dell Inspiron 3000 Core i7 9th Gen - (8 GB/512...,"₹93,490",intel core i7 processor (9th gen)8 gb ddr4 ram...,2.7,No reviews for this Laptop
7,Dell Inspiron Core i3 10th Gen - (4 GB/1 TB HD...,"₹62,990",intel core i3 processor (10th gen)4 gb ddr4 ra...,4.5,Too good laptop look wise performance wise ver...
8,Dell Inspiron Core i5 10th Gen - (8 GB/512 GB ...,"₹86,990",intel core i5 processor (10th gen)8 gb ddr4 ra...,0.0,No reviews for this Laptop
9,Dell Vostro 3000 Core i5 8th Gen - (8 GB/1 TB ...,"₹51,130",intel core i5 processor (8th gen)8 gb ddr4 ram...,4.1,this is good product. i got it in sell at quit...


In [6]:

## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

dataset["Features"] = dataset["Features"].apply(lambda text: remove_punctuation(text))

In [16]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


dataset["Features"] = dataset["Features"].apply(lambda text: remove_stopwords(text))

In [15]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
def get_top_words(column, top_nu_of_words, nu_of_word):
    
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    
    bag_of_words = vec.fit_transform(column)
    
    sum_words = bag_of_words.sum(axis=0)
    
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    return words_freq[:top_nu_of_words]


In [18]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(dataset['Features'])


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [21]:

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:

dataset.set_index('Product Name', inplace=True)

In [25]:
indices = pd.Series(dataset.index)

In [26]:

def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top 10 restaurants
    recommend_laptop = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:5].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_laptop.append(list(dataset.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['Price', 'Rating', 'Review'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_laptop:
        df_new = dataset.append(pd.DataFrame(dataset[['Price', 'Rating', 'Review']][dataset.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['Price', 'Rating', 'Review'], keep=False)
    df_new = df_new.sort_values(by='Rating', ascending=False).head(10)
    
    print('TOP %s LAPTOPS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new


In [27]:

recommend('Dell Inspiron 15 7000 Series Core i5 8th Gen - (8 GB/1 TB HDD/128 GB SSD/Windows 10 Home/2 GB Graphics...')

TOP 10 LAPTOPS LIKE Dell Inspiron 15 7000 Series Core i5 8th Gen - (8 GB/1 TB HDD/128 GB SSD/Windows 10 Home/2 GB Graphics... WITH SIMILAR REVIEWS: 


,Price,Features,Rating,Review
Product Name,,,,
Dell G7 15 7000 Series Core i9 8th Gen - (16 GB/1 TB HDD/128 GB SSD/Windows 10 Home/6 GB Graphics/NVID...,"₹1,38,945",intel core i9 processor 8th gen16 gb ddr4 ram6...,4.7,No reviews for this Laptop
Dell Inspiron 15 5000 Series Core i5 8th Gen - (4 GB + 16 GB Optane/2 TB HDD/Windows 10 Home/2 GB Grap...,"₹64,890",intel core i5 processor 8th gen4 gb ddr4 ram64...,4.6,"very good looking ,very slimREAD MORE"
Dell Inspiron Core i7 7th Gen - (16 GB/1 TB HDD/256 GB SSD/Windows 10 Home/4 GB Graphics/NVIDIA Geforc...,"₹1,19,940",preloaded microsoft office home student 2016pr...,4.5,The Laptop config I purchased was: Intel i7 7t...
Dell Inspiron Core i3 10th Gen - (4 GB/1 TB HDD/256 GB SSD/Windows 10 Home) Inspiron 5491 2 in 1 Lapto...,"₹62,990",intel core i3 processor 10th gen4 gb ddr4 ram6...,4.5,Too good laptop look wise performance wise ver...
Dell G3 Core i7 9th Gen - (8 GB/1 TB HDD/512 GB SSD/Windows 10 Home/4 GB Graphics/NVIDIA Geforce GTX 1...,"₹84,990",intel core i7 processor 9th gen8 gb ddr4 ram64...,4.4,Display is 220nits bright( Sufficient for dail...
Dell G3 Series Core i5 8th Gen - (8 GB/1 TB HDD/128 GB SSD/Windows 10 Home/4 GB Graphics/NVIDIA Geforc...,"₹66,990",60 hz refresh rate display upto 60 frames per ...,4.4,"it's good , it's performance is better than my..."
Dell Inspiron 15 5000 Core i7 8th Gen - (8 GB/2 TB HDD/128 GB SSD/Windows 10 Home/4 GB Graphics) 5570 ...,"₹81,989",intel core i7 processor 8th gen8 gb ddr4 ram64...,4.3,purchased at 75k.price to performance ratio(8t...
Dell Inspiron 5000 Core i3 10th Gen - (4 GB/256 GB SSD/Windows 10 Home) 5491 2 in 1 Laptop,"₹57,890",intel core i3 processor 10th gen4 gb ddr4 ram6...,4.3,Excellent product for general computing ( MS o...
Dell Vostro Core i3 10th Gen - (4 GB/1 TB HDD/256 GB SSD/Windows 10 Home) Vostro 3491 Thin and Light L...,"₹39,998",intel core i3 processor 10th gen4 gb ddr4 ram6...,4.3,After 1 week of usage the laptop performance i...
